In [1]:
import os
import PIL.Image
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

from helpers.custom_classifier import CustomClassifier
from helpers.early_stopping import EarlyStopping

In [2]:
num_epochs = 25

In [3]:
class CrackDataset(Dataset):
    def __init__(self, images_dir, transform: transforms.Compose):
        self.images_dir = images_dir
        self.image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, int]:
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = PIL.Image.open(img_path).convert("RGB")
        label = 0 if "noncrack" in img_name else 1
        image = self.transform(image)

        return image, label

In [4]:
def get_loaders() -> tuple[DataLoader, DataLoader]:
    batch_size = 32
    train_images_dir = os.path.join("data", "train", "images")
    valid_images_dir = os.path.join("data", "valid", "images")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    train_dataset = CrackDataset(train_images_dir, transform=transform)
    valid_dataset = CrackDataset(valid_images_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, valid_loader

In [5]:
def get_loop_objects(
        conv_out_shapes: tuple[int, int],
        linear_layers_features: int
) -> tuple[CustomClassifier, EarlyStopping, torch.nn.BCEWithLogitsLoss, optim.Adam, torch.device]:
    model = CustomClassifier(conv_out_shapes=conv_out_shapes, linear_layers_features=linear_layers_features)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    early_stopping = EarlyStopping(patience=7, verbose=True, delta=0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    
    return model, early_stopping, criterion, optimizer, device

In [6]:
def run_training_loop(
        conv_out_shapes: tuple[int, int], 
        linear_layers_features: int,
        checkpoint_path: str
) -> dict:
    train_loader, valid_loader = get_loaders()
    model, early_stopping, criterion, optimizer, device = get_loop_objects(conv_out_shapes, linear_layers_features)
    history = {
        "train_loss": [],
        "valid_loss": []
    }
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
    
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
    
            for images, labels in tepoch:
                images, labels = images.to(device), labels.to(device).float()
    
                optimizer.zero_grad()
    
                # squeeze because the outputs are (BATCH_SIZE, 1) shape, and should be of (BATCH_SIZE,) shape
                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels)
    
                loss.backward()
                optimizer.step()
    
                train_loss += loss.item()
                predicted = (outputs > 0.5).float()
                correct_train += predicted.eq(labels).sum().item()
                total_train += labels.size(0)
    
                tepoch.set_postfix(loss=train_loss/total_train, accuracy=100.*correct_train/total_train)
    
        train_loss /= len(train_loader.dataset)
        history["train_loss"].append(train_loss)
    
        model.eval()
    
        valid_loss = 0.0
        correct_valid = 0
        total_valid = 0
    
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device).float()
                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                predicted = (outputs > 0.5).float()
                correct_valid += predicted.eq(labels).sum().item()
                total_valid += labels.size(0)
    
        valid_loss /= len(valid_loader.dataset)
        history["valid_loss"].append(valid_loss)
    
        valid_accuracy = 100. * correct_valid / total_valid
    
        print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%")
        early_stopping(valid_loss, model, path=checkpoint_path)
    
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
            
    return history

In [7]:
model_results = []
param_combinations = [
    {
        "conv_out_shapes": (64, 128),
        "linear_layers_features": 256,
    },
    {
        "conv_out_shapes": (128, 256),
        "linear_layers_features": 512,
    },
    {
        "conv_out_shapes": (64, 128),
        "linear_layers_features": 512,
    },
    {
        "conv_out_shapes": (64, 128),
        "linear_layers_features": 1024,
    },
    {
        "conv_out_shapes": (128, 256),
        "linear_layers_features": 1024,
    },
]

for param_combination in param_combinations:
    model_custom_path = "_".join(map(str, param_combination["conv_out_shapes"]))
    model_custom_path = f"{param_combination['linear_layers_features']}_{model_custom_path}"
    checkpoint_path = os.path.join("checkpoints", f"custom_classifier_{model_custom_path}.pt")
    history = run_training_loop(
        param_combination["conv_out_shapes"], 
        param_combination["linear_layers_features"], 
        checkpoint_path
    )
    
    model_results.append((checkpoint_path, history))
    torch.cuda.empty_cache()

Epoch 1/25: 100%|██████████| 301/301 [01:29<00:00,  3.38batch/s, accuracy=91.9, loss=0.00678]


Validation Loss: 0.0063, Validation Accuracy: 93.33%
Validation loss decreased (inf --> 0.006259).  Saving model ...


Epoch 2/25: 100%|██████████| 301/301 [01:18<00:00,  3.83batch/s, accuracy=93.5, loss=0.00572]


Validation Loss: 0.0058, Validation Accuracy: 91.09%
Validation loss decreased (0.006259 --> 0.005835).  Saving model ...


Epoch 3/25: 100%|██████████| 301/301 [01:18<00:00,  3.83batch/s, accuracy=93.6, loss=0.00532]


...

Validation Loss: 0.0053, Validation Accuracy: 96.17%
EarlyStopping counter: 6 out of 7


Epoch 12/25: 100%|██████████| 301/301 [01:43<00:00,  2.91batch/s, accuracy=99.3, loss=0.000648]


Validation Loss: 0.0056, Validation Accuracy: 95.10%
EarlyStopping counter: 7 out of 7
Early stopping triggered


Epoch 1/25:   0%|          | 0/301 [00:01<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.06 GiB. GPU 0 has a total capacity of 12.00 GiB of which 627.00 MiB is free. Of the allocated memory 9.23 GiB is allocated by PyTorch, and 54.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
for key, (_, history) in model_results.items():
    plt.figure(figsize=(10, 5))
    plt.plot(history["train_loss"], label="Training Loss")
    plt.plot(history["valid_loss"], label="Validation Loss")
    plt.title("Training vs Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()